In [11]:
from hotels_50k import Hotels50KDataset

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from torchvision import datasets

In [22]:
dataset = Hotels50KDataset(root='../hotels50k', target='chains', download=False)

Loading image folders
Concatting dataset
Getting labels


In [25]:
dataset.labels.shape

(1027756,)

In [28]:
len(dataset.original_train_dataset.targets), len(dataset.original_test_dataset.targets)

(1011587, 16169)

In [43]:
import numpy as np

In [44]:
np.where(dataset.labels==1)

(array([ 582807,  582808,  582809, ..., 1015674, 1015675, 1015676]),)

In [48]:
assert list(dataset.labels[582807:582817]) == [dataset[i][1] for i in range(582807, 582817)]

In [60]:
from powerful_benchmarker.split_managers import ClassDisjointSplitManager, BaseSplitManager, IndexSplitManager
from torchvision import transforms
import torch
import numpy as np

class CustomSplitManager(BaseSplitManager):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.split_names = ["test"]

    def _create_split_schemes(self, datasets):
        output = {}
        for transform_type, v1 in datasets.items():
            output[transform_type] = {}
            for split_name, v2 in v1.items():
                indices = v2.get_split_indices(split_name)
                if indices is not None:
                    output[transform_type][split_name] = torch.utils.data.Subset(v2, indices)
                else:
                    output[transform_type][split_name] = v2
        return {self.get_split_scheme_name(0): output}

    def get_test_set_name(self):
        return 'UsingOriginalTest'

    def get_base_split_scheme_name(self):
        return self.get_test_set_name()

    def get_split_scheme_name(self, partition):
        return self.get_base_split_scheme_name()

    def split_assertions(self):
        pass


dataset_root = '../hotels50k'
the_dataset = Hotels50KDataset

ds = the_dataset(dataset_root)

# first level of dictionary is transform type
# second level consists of separate dataset objects for each split
datasets = {"eval": {"train": ds,
                    "val": ds,
                    "test": ds}
            }

helper_split_manager = CustomSplitManager()
split_manager = IndexSplitManager(num_training_partitions=4, num_training_sets=4, helper_split_manager=helper_split_manager)

split_manager.create_split_schemes(datasets)

test_set = split_manager.get_dataset("eval", "test")

for name in split_manager.split_scheme_names:
    split_manager.set_curr_split_scheme(name)
    print("CURR SPLIT SCHEME =", split_manager.curr_split_scheme_name)
    print("train len", len(split_manager.get_dataset("eval", "train")))
    print("val len", len(split_manager.get_dataset("eval", "val")))
    print("test len", len(split_manager.get_dataset("eval", "test")))

    print("train labels", split_manager.get_label_set("eval", "train"))
    print("val labels", split_manager.get_label_set("eval", "val"))
    print("test labels", split_manager.get_label_set("eval", "test"))
    
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "val").indices) == set()
    assert set(split_manager.get_dataset("eval", "train").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()
    assert set(split_manager.get_dataset("eval", "val").indices).intersection(split_manager.get_dataset("eval", "test").indices) == set()


Loading image folders
Concatting dataset
Getting labels
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_0
train len 758690
val len 252897
test len 16169
train labels {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92}
val labels {0}
test labels {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88}
CURR SPLIT SCHEME = UsingOriginalTest_Partitions4_1
train len 758690
val len 252897
test len 16